# Nebius Managed Service for MLflow Demo

This demo shows you how key use cases of using the Nebius Managed Service for MLflow for AI development.

## Getting Started

1. Launch your instance of the Managed Service for MLflow with [the MLflow quickstart](https://docs.nebius.com/mlflow/quickstart).
2. Set up your API key to connect to Nebius AI Studio with [the AI Studio quickstart](https://docs.nebius.com/studio/inference/quickstart).

> **Note:** Launching an MLflow cluster may take 30-60 minutes to be fully provisioned and ready to use.

In [1]:
!pip install mlflow==2.20.2 python-dotenv openai

### Secrets and Environment Variables

You will need to set the following environment variables where this notebook is running, so that the code in the following cells can connect to both Nebius Managed Service for MLflow and Nebius AI Studio. 

MLflow:<br>
`MLFLOW_TRACKING_SERVER_CERT_PATH`<br>
`MLFLOW_TRACKING_URI`<br>
`MLFLOW_TRACKING_USERNAME`<br>
`MLFLOW_TRACKING_PASSWORD`<br>

AI Studio:<br>
`NEBIUS_API_KEY`

### Environment Setup

To set the environment variables, run the following cell. You may choose to set them interactively or by loading from a `.env` file.

In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Add the parent directory to Python path
sys.path.append(str(Path.cwd().parent))

from env_setup import setup_env_from_file, setup_env_interactive, verify_env_setup

# Option 1: Interactive setup
# setup_env_interactive()

# Option 2: Load from .env file
setup_env_from_file('../.env')

# Verify the setup
verify_env_setup()

Load ENV variables from file: ../.env
All required environment variables are set.


True

### Check connection to MLflow


In [2]:
import mlflow 

# List experiments in MLflow
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1744716009071, experiment_id='2', last_update_time=1744716009071, lifecycle_stage='active', name='Autolog', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1744712739337, experiment_id='1', last_update_time=1744712739337, lifecycle_stage='active', name='CrewAI Agent Debugging Demo', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1744710198517, experiment_id='0', last_update_time=1744710198517, lifecycle_stage='active', name='Default', tags={}>]

### Set up Nebius AI Studio client

In [3]:
import openai

API_KEY = os.environ.get("NEBIUS_API_KEY")

# Instantiate the client instance
nebius_client = openai.OpenAI(api_key=API_KEY,
                              base_url="https://api.studio.nebius.ai/v1/")


# Example 1: MLflow Tracking Quickstart

https://mlflow.org/docs/latest/getting-started/intro-quickstart/

In [21]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

/home/jovyan/.conda/envs/default/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


### Log the model and its metadata to MLflow

- Initiate an MLflow run context to start a new run that we will log the model and metadata to.
- Log model parameters and performance metrics.
- Tag the run for easy retrieval.
- Register the model in the MLflow Model Registry while logging (saving) the model.


In [22]:
# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2025/03/25 11:59:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 4
Created version '4' of model 'tracking-quickstart'.


🏃 View run colorful-auk-46 at: https://public-tracking.mlflow-e00e997y0znyf15mwg.backbone-e00rt16p71ew33dg9e.msp.eu-north1.nebius.cloud/#/experiments/1/runs/263bbfe1281f4a0bb5e60cedee2d2523
🧪 View experiment at: https://public-tracking.mlflow-e00e997y0znyf15mwg.backbone-e00rt16p71ew33dg9e.msp.eu-north1.nebius.cloud/#/experiments/1


In [23]:
model_info.model_uri

'runs:/263bbfe1281f4a0bb5e60cedee2d2523/iris_model'

## Load the model as a Python Function (pyfunc) and use it for inference

- Loading the model using MLflow's pyfunc flavor.
- Running Predict on new data using the loaded model

In [24]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: iris_model
  flavor: mlflow.sklearn
  run_id: 263bbfe1281f4a0bb5e60cedee2d2523

In [25]:
# Get predictions 

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1


# Example 2: MLflow Tracing for LLM Observability

Traces enhances LLM observability in your Generative AI (GenAI) applications by capturing detailed information about the execution details. 

https://mlflow.org/docs/latest/tracing/

## Automatic Tracing of LLM calls

In [35]:
!env | grep MLFLOW_TRACKING_SERVER_CERT_PATH



MLFLOW_TRACKING_SERVER_CERT_PATH=/home/jovyan/ca.pem


In [6]:
import mlflow
import openai

tracing_experiment =mlflow.set_experiment("MLflow Tracing")

# Enable MLflow automatic tracing for OpenAI with one line of code!
mlflow.openai.autolog()


# Time to call the LLM -- tracing is done automatically
nebius_client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct",
    temperature=0.95,
    messages=[
        {"role": "system", "content": "You are a chatbot."},
        {"role": "user", "content": "What is the weather like today?"},
    ],
)


ChatCompletion(id='chatcmpl-508058828bc24bd7b2eef9e48eea3ae1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm just a chatbot, I don't have have access to real-time information or a physical location, so I don't know the current weather. But I can suggest some ways for you to find out the weather:\n\n1. Check online weather websites or apps, such as AccuWeather or Weather.com, for the current weather conditions in your area.\n2. Look out the window to see the current weather conditions.\n3. Check your local news or weather forecast for updates on the weather.\n\nIf you'd like to know the weather for a specific location, please let me know and I can try to provide you with general information about the typical weather patterns for that area.", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1742904463, model='meta-llama/Llama-3.3-

Trace(request_id=3aad225a92ad464288dd0d3268b86321)

## Manual Tracing

1. Instrument a function with @mlflow.trace decorator.
2. Instrument any block of code using mlflow.start_span context manager.
3. Grouping or annotating traces using a tag.
4. Disabling trace globally.

https://mlflow.org/docs/latest/tracing/#manual-tracing

In [7]:
import mlflow
from mlflow.entities import SpanType
import openai
import time

In [8]:
# Use MLflow tracing to trace the execution of a function

@mlflow.trace(span_type="func", attributes={"key": "value"})
def add_1(x):
    return x + 1


@mlflow.trace(span_type="func", attributes={"key": "value"})
def minus_1(x):
    return x - 1


@mlflow.trace(name="Trace Test")
def trace_test(x):
    step1 = add_1(x)
    return minus_1(step1)


trace_test(4)

4

Trace(request_id=b558f8929bd245ebba7fc39fa43ecaf5)

In [11]:
# Integrate tracing into your LLM workflow

mlflow.openai.autolog()

@mlflow.trace(span_type=SpanType.CHAIN)
def run(question):
    messages = build_messages(question)
    # MLflow automatically generates a span for OpenAI invocation
    response = nebius_client.chat.completions.create(
        # model="gpt-4o-mini",
        model="meta-llama/Llama-3.3-70B-Instruct",
        max_tokens=100,
        messages=messages,
    )
    return parse_response(response)


@mlflow.trace
def build_messages(question):
    return [
        {"role": "system", "content": "You are a helpful chatbot."},
        {"role": "user", "content": question},
    ]

@mlflow.trace
def parse_response(response):
    return response.choices[0].message.content


run("What is MLflow?")

'**MLflow** is an open-source platform for managing the end-to-end machine learning (ML) lifecycle. It helps data scientists and engineers to track, reproduce, and deploy ML models efficiently. MLflow provides a standard way to manage ML workflows, including data preparation, model training, model evaluation, and model deployment.\n\n### Key Features of MLflow\n\n1. **Experiment Tracking**: MLflow allows users to track and manage experiments, including hyperparameter tuning, model selection, and results comparison.\n2'

Trace(request_id=c20f4e1ea4f34caeb7cdd2a51c2dc459)

In [12]:
# Get the timestamp in milliseconds
one_hour_ago = int(time.time() - 3600) * 1000  # 3600 seconds = 1 hour in milliseconds

one_hour_ago

1742901029000

In [13]:
# Search and analyze traces

mlflow.search_traces(
    tracing_experiment.experiment_id, 
    filter_string=f"timestamp_ms < {one_hour_ago}",
)[:3]

,request_id,trace,timestamp_ms,status,execution_time_ms,request,response,request_metadata,spans,tags
0,0383cf8068844fa6a9245fdc233aabd5,Trace(request_id=0383cf8068844fa6a9245fdc233aa...,1742854437706,TraceStatus.OK,3088,"{'kwargs': {'provider': 'huggingface-hub', 'co...","{'config': {'path': 'intfloat/e5-base', 'instr...","{'mlflow.trace_schema.version': '2', 'mlflow.t...","[{'name': 'Embeddings.load', 'context': {'span...",{'mlflow.source.name': '/Users/mikhailrozhkov/...
1,9969d703bbe84fdba40f8edfe4fce715,Trace(request_id=9969d703bbe84fdba40f8edfe4fce...,1742853900983,TraceStatus.OK,3869,"{'kwargs': {'provider': 'huggingface-hub', 'co...","{'config': {'autoid': 'uuid5', 'path': 'intflo...","{'mlflow.trace_schema.version': '2', 'mlflow.t...","[{'name': 'Embeddings.load', 'context': {'span...",{'mlflow.source.name': '/Users/mikhailrozhkov/...
2,776d1d9d3a204c41975729429a535faf,Trace(request_id=776d1d9d3a204c41975729429a535...,1742853720840,TraceStatus.OK,3595,"{'kwargs': {'provider': 'huggingface-hub', 'co...","{'config': {'autoid': 'uuid5', 'path': 'intflo...","{'mlflow.trace_schema.version': '2', 'mlflow.t...","[{'name': 'Embeddings.load', 'context': {'span...",{'mlflow.source.name': '/Users/mikhailrozhkov/...


[Trace(request_id=0383cf8068844fa6a9245fdc233aabd5), Trace(request_id=9969d703bbe84fdba40f8edfe4fce715), Trace(request_id=776d1d9d3a204c41975729429a535faf), Trace(request_id=a661fda04e7145dba4fbac229f650c3a), Trace(request_id=278c58f89114413d86bca0da2d9cdf54), Trace(request_id=63e9174f3feb4763a478567d3dd0b924), Trace(request_id=ffd322293a8f4d4d91aecb501f77a98d), Trace(request_id=fe0d01252772434a98f5a0a05f5acad2), Trace(request_id=e23de319f2344493a5261fd9b55664d6), Trace(request_id=9536954c27f145339f90cba23380c3d3)]

# Example 3:  Tracing LangGraph 

- Tracing LangGraph with MLflow https://mlflow.org/docs/latest/tracing/integrations/langgraph 
- Example: Code generation with RAG and self-correction with https://langchain-ai.github.io/langgraph/tutorials/code_assistant/langgraph_code_assistant/

For this example set additional vars: 
- OPENAI_API_KEY
- ANTHROPIC_API_KEY

In [5]:
!pip install langchain_openai langchain langgraph langchain_core

  Using cached langgraph-0.3.21-py3-none-any.whl.metadata (7.7 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
Using cached langgraph-0.3.21-py3-none-any.whl (138 kB)
Using cached langsmith-0.1.147-py3-none-any.whl (311 kB)
  Attempting uninstall: xxhash
    Found existing installation: xxhash 3.4.1
    Uninstalling xxhash-3.4.1:
      Successfully uninstalled xxhash-3.4.1
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.93
    Uninstalling langsmith-0.1.93:
      Successfully uninstalled langsmith-0.1.93
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.2.23
    Uninstalling langchain-core-0.2.23:
      Successfully uninstalled langchain-core-0.2.23


In [6]:
from typing import Literal

import mlflow

from langchain_core.messages import AIMessage, ToolCall
from langchain_core.outputs import ChatGeneration, ChatResult
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# Enabling tracing for LangGraph (LangChain)
mlflow.langchain.autolog()

# Optional: Set a tracking URI and an experiment
# mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("LangGraph")


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"


llm = ChatOpenAI(model="gpt-4o-mini")
tools = [get_weather]
graph = create_react_agent(llm, tools)

# Invoke the graph
result = graph.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in tokyo?"}]}
)

Trace(request_id=7cce8f1752ad494292a73947ddd4d281)

# Example 4: Tracing CrewAI Agents

- Example https://mlflow.org/docs/latest/tracing/integrations/crewai 

In [4]:
!pip install crewai crewai_tools

In [5]:
import mlflow

# Turn on auto tracing by calling mlflow.crewai.autolog()
mlflow.crewai.autolog()

mlflow.set_experiment("CrewAI")

2025/04/15 13:28:13 INFO mlflow.tracking.fluent: Experiment with name 'CrewAI' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1744716493013, experiment_id='3', last_update_time=1744716493013, lifecycle_stage='active', name='CrewAI', tags={}>

2025-04-15 21:10:15,569 - 13915959296 - __init__.py-__init__:364 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/urllib3/connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/http/client.py", line 1378, in getresponse
    response.begin()
  File "/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/http/client.py", line 318,

In [6]:
from crewai import Agent, Crew, Task
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
from crewai_tools import SerperDevTool, WebsiteSearchTool

from textwrap import dedent

content = "Users name is John. He is 30 years old and lives in San Francisco."
string_source = StringKnowledgeSource(
    content=content, metadata={"preference": "personal"}
)

search_tool = WebsiteSearchTool()

class TripAgents:
    def city_selection_agent(self):
        return Agent(
            role="City Selection Expert",
            goal="Select the best city based on weather, season, and prices",
            backstory="An expert in analyzing travel data to pick ideal destinations",
            tools=[
                search_tool,
            ],
            verbose=True,
        )

    def local_expert(self):
        return Agent(
            role="Local Expert at this city",
            goal="Provide the BEST insights about the selected city",
            backstory="""A knowledgeable local guide with extensive information
        about the city, it's attractions and customs""",
            tools=[search_tool],
            verbose=True,
        )

class TripTasks:
    def identify_task(self, agent, origin, cities, interests, range):
        return Task(
            description=dedent(
                f"""
                Analyze and select the best city for the trip based
                on specific criteria such as weather patterns, seasonal
                events, and travel costs. This task involves comparing
                multiple cities, considering factors like current weather
                conditions, upcoming cultural or seasonal events, and
                overall travel expenses.
                Your final answer must be a detailed
                report on the chosen city, and everything you found out
                about it, including the actual flight costs, weather
                forecast and attractions.

                Traveling from: {origin}
                City Options: {cities}
                Trip Date: {range}
                Traveler Interests: {interests}
            """
            ),
            agent=agent,
            expected_output="Detailed report on the chosen city including flight costs, weather forecast, and attractions",
        )

    def gather_task(self, agent, origin, interests, range):
        return Task(
            description=dedent(
                f"""
                As a local expert on this city you must compile an
                in-depth guide for someone traveling there and wanting
                to have THE BEST trip ever!
                Gather information about key attractions, local customs,
                special events, and daily activity recommendations.
                Find the best spots to go to, the kind of place only a
                local would know.
                This guide should provide a thorough overview of what
                the city has to offer, including hidden gems, cultural
                hotspots, must-visit landmarks, weather forecasts, and
                high level costs.
                The final answer must be a comprehensive city guide,
                rich in cultural insights and practical tips,
                tailored to enhance the travel experience.

                Trip Date: {range}
                Traveling from: {origin}
                Traveler Interests: {interests}
            """
            ),
            agent=agent,
            expected_output="Comprehensive city guide including hidden gems, cultural hotspots, and practical travel tips",
        )


class TripCrew:
    def __init__(self, origin, cities, date_range, interests):
        self.cities = cities
        self.origin = origin
        self.interests = interests
        self.date_range = date_range

    def run(self):
        agents = TripAgents()
        tasks = TripTasks()

        city_selector_agent = agents.city_selection_agent()
        local_expert_agent = agents.local_expert()

        identify_task = tasks.identify_task(
            city_selector_agent,
            self.origin,
            self.cities,
            self.interests,
            self.date_range,
        )
        gather_task = tasks.gather_task(
            local_expert_agent, self.origin, self.interests, self.date_range
        )

        crew = Crew(
            agents=[city_selector_agent, local_expert_agent],
            tasks=[identify_task, gather_task],
            verbose=True,
            memory=True,
            knowledge={
                "sources": [string_source],
                "metadata": {"preference": "personal"},
                "collection_name":"knowledge",
            },
        )

        result = crew.kickoff()
        return result


trip_crew = TripCrew("California", "Tokyo", "Dec 12 - Dec 20", "sports")


/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/pydantic/_internal/_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/paramiko/transport.py:258: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitiv

In [7]:
# Run the crew

result = trip_crew.run()
result.dict()

2025/04/15 13:28:20 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/pydantic/fields.py:783: PydanticDeprecatedSince20: `min_items` is deprecated and will be removed, use `min_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/"


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 16774dcb-af73-45b7-918f-96e6e66a9c6d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: f7728a80-539d-4518-bb51-6084327eeb9d
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: f7728a80-539d-4518-bb51-6084327eeb9d
       Status: Executing Task...
    └── 🤖 Agent: City Selection Expert
            Status: In Progress

# Agent: City Selection Expert
## Task: 
Analyze and select the best city for the trip based
on specific criteria such as weather patterns, seasonal
events, and travel costs. This task involves comparing
multiple cities, considering factors like current weather
conditions, upcoming cultural or seasonal events, and
overall travel expenses.
Your final answer must be a detailed
report on the chosen city, and everything you found out
about it, including the actual flight costs, weather
forecast and attractions.

Traveling from: California
City Options: Tokyo
Trip Date: Dec 12 - Dec 20
Traveler Interests: sports



...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: c10185bb-f6d2-4fcf-9904-aff32727a333                                                                     │
│  Agent: Local Expert at this city                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 16774dcb-af73-45b7-918f-96e6e66a9c6d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


{'raw': "# Comprehensive City Guide to Tokyo, Japan (December 12 - December 20, 2023)\n\n---\n\n**1. Arrival and Flight Costs:**\nTraveling from California, the average round-trip flight price from San Francisco to Tokyo is approximately **$900**. Major airlines such as **United**, **ANA**, and **American Airlines** have routes available. Booking early can secure better prices, and keeping flexibility in dates can help find lower tickets.\n\n---\n\n**2. Weather Forecast:**\nTokyo typically experiences mild but cold weather in December. Here’s the forecast to help you pack accordingly:\n- **December 12:** High of 15°C (59°F), Low of 7°C (45°F), Mostly Sunny\n- **December 13:** High of 14°C (57°F), Low of 8°C (46°F), Partly Cloudy\n- **December 14:** High of 14°C (57°F), Low of 6°C (43°F), Rain Expected\n- **December 15:** High of 12°C (54°F), Low of 5°C (41°F), Showers Possible\n- **December 16:** High of 13°C (55°F), Low of 6°C (43°F), Mostly Sunny\n- **December 17:** High of 11°C (52°

Trace(request_id=9be3dec58ea64cab834eb3a7a34eb1ad)

## Updated example 

In [5]:
!pip install crewai==0.108.0 crewai-tools duckduckgo-search langchain-openai

In [18]:
import os
from textwrap import dedent
from crewai import Agent, Crew, Task, Process
from crewai_tools import WebsiteSearchTool, DuckDuckGoSearchRunTool
# You might need specific LangChain components depending on your LLM setup
from langchain_openai import ChatOpenAI # Example for OpenAI or compatible APIs like Ollama

print("Libraries imported.")


ImportError: cannot import name 'DuckDuckGoSearchRunTool' from 'crewai_tools' (/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/crewai_tools/__init__.py)

In [16]:
# Turn on auto tracing by calling mlflow.crewai.autolog()
mlflow.crewai.autolog()

mlflow.set_experiment("CrewAI Agents")

2025/03/29 20:39:42 INFO mlflow.tracking.fluent: Experiment with name 'CrewAI Agents' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/12', creation_time=1743277182597, experiment_id='12', last_update_time=1743277182597, lifecycle_stage='active', name='CrewAI Agents', tags={}>

In [8]:
# --- VERY IMPORTANT: Configure Your LLM ---
# Option A: Set OpenAI API Key as Environment Variable (Recommended for Simplicity)
# Make sure your OPENAI_API_KEY environment variable is set *before* launching Jupyter/VSCode.
# CrewAI will automatically pick it up if no specific 'llm' is passed to Agents/Crew.
# You can uncomment the line below to set it temporarily for this session *only*
# os.environ["OPENAI_API_KEY"] = "sk-YOUR_API_KEY_HERE"

# Option B: Configure for a Local or Specific LLM (Example using Ollama via ChatOpenAI)
# Ensure Ollama is running (e.g., `ollama run llama3`)
# ollama_llm = ChatOpenAI(
#     model="llama3", # Or whichever model you are running in Ollama
#     base_url="http://localhost:11434/v1",
#     api_key="NA" # Standard practice for Ollama via langchain-openai
# )
# print("LLM Configuration (Example - Ollama): Set up.")
# If using Option B, you'll need to pass `llm=ollama_llm` when creating Agents below.

# Check if the key is set (optional check for Option A)
api_key = os.environ.get("OPENAI_API_KEY")
if api_key:
    print("OPENAI_API_KEY found. Agents will use the default OpenAI LLM unless specified otherwise.")
else:
    print("WARNING: OPENAI_API_KEY environment variable not found.")
    print("Ensure an LLM is configured either via environment variables or by passing an 'llm' object to Agents.")

OPENAI_API_KEY found. Agents will use the default OpenAI LLM unless specified otherwise.


## Setup WebsiteSearchTool

In [9]:
# --- Cell 3: Instantiate Tools ---
search_tool = WebsiteSearchTool()
# If you were using Serper:
# from crewai_tools import SerperDevTool
# os.environ["SERPER_API_KEY"] = "YOUR_SERPER_KEY" # Set environment variable
# search_tool = SerperDevTool()

print("Tools instantiated (WebsiteSearchTool).")

Tools instantiated (WebsiteSearchTool).


In [10]:
# --- Cell 4: Define Agent Creation Class ---
class MLflowIntegrationAgents:
    def trend_scout_agent(self, llm_config=None): # Pass LLM config if not relying on default
        return Agent(
            role="AI Technology Trend Scout",
            goal=dedent(
                """Identify a specific, promising new tool, library, or technique
                within a given area of AI/ML technology."""
            ),
            backstory=dedent(
                """An expert researcher constantly scanning the horizon
                for emerging AI/ML technologies, publications, and popular
                open-source projects. You prioritize novelty and potential impact."""
            ),
            tools=[search_tool],
            allow_delegation=False,
            verbose=True,
            llm=llm_config # Pass the specific LLM object here if needed
        )

    def technical_analyst_agent(self, llm_config=None): # Pass LLM config if not relying on default
        return Agent(
            role="AI Technical Analyst",
            goal=dedent(
                """Analyze the technical capabilities, architecture, pros, cons,
                and primary use cases of a specific AI tool or technique."""
            ),
            backstory=dedent(
                """A meticulous engineer who dives deep into documentation,
                tutorials, and technical blogs to understand how technologies
                work under the hood. You focus on practical implementation details."""
            ),
            tools=[search_tool],
            allow_delegation=False,
            verbose=True,
            llm=llm_config # Pass the specific LLM object here if needed
        )

    def mlflow_integration_assessor_agent(self, llm_config=None): # Pass LLM config if not relying on default
        return Agent(
            role="MLflow Integration Assessor",
            goal=dedent(
                """Assess how a given AI tool/technique could be integrated
                into a standard MLflow workflow. Identify potential logging points,
                artifact types, customizability needs, or existing MLflow plugins."""
            ),
            backstory=dedent(
                """A seasoned MLOps engineer with deep expertise in MLflow.
                You understand the ML lifecycle and how various components
                can be tracked and managed using MLflow runs, artifacts, models,
                and parameters. You think practically about integration points."""
            ),
            tools=[search_tool], # May need search for finding MLflow plugins
            allow_delegation=False,
            verbose=True,
            llm=llm_config # Pass the specific LLM object here if needed
        )

print("MLflowIntegrationAgents class defined.")
# Example instantiation check (if using Option B from Cell 2):
# agent_creator = MLflowIntegrationAgents()
# scout = agent_creator.trend_scout_agent(llm_config=ollama_llm)
# print(scout) # Check agent creation

MLflowIntegrationAgents class defined.


In [11]:
# --- Cell 5: Define Task Creation Class ---
class MLflowIntegrationTasks:
    # Task 1: No context needed as it's the first step
    def scout_task(self, agent, area_of_interest):
        return Task(
            description=dedent(
                f"""
                Identify one specific, noteworthy, and relatively new open-source tool,
                library, or technique related to '{area_of_interest}'.
                Provide its name and a brief (1-2 sentence) description of what it does.
                Focus on things gaining traction or offering novel capabilities.
                Avoid general concepts; find a concrete example.

                Example Areas: Vector Databases, LLM Serving Frameworks,
                               Data Versioning for ML, Explainable AI Libraries.

                Your final output MUST be the name of the tool/technique and the brief description ONLY.
            """
            ),
            agent=agent,
            expected_output="The name of a specific tool/library/technique and a 1-2 sentence description.",
            # cache=True # Optional: Cache the output of this task
        )

    # Task 2: Depends on the output of scout_task
    def analyze_task(self, agent, area_of_interest, context_task): # Added context_task parameter
        return Task(
            description=dedent(
                f"""
                Based on the tool/technique identified in the previous step for the area '{area_of_interest}',
                perform a technical analysis. Research its core features, how it works
                (high-level architecture if possible), main benefits, and potential drawbacks or limitations.
                Use web search to find its documentation, tutorials, or technical articles.

                Your final output must be a bulleted list summarizing:
                - Core Features
                - How it Works (Briefly)
                - Key Benefits
                - Potential Drawbacks/Limitations
            """
            ),
            agent=agent,
            expected_output="Bulleted list summarizing features, workings, benefits, and drawbacks.",
            context=[context_task] # Explicitly state dependency on the previous task
            # cache=True # Optional: Cache the output of this task
        )

    # Task 3: Depends on the output of analyze_task
    def assess_mlflow_integration_task(self, agent, context_task): # Added context_task parameter
        return Task(
            description=dedent(
                f"""
                Considering the identified tool/technique and its technical analysis from the previous steps:
                Assess how this tool/technique could be integrated with or tracked by MLflow.
                Think about the typical ML lifecycle (data prep, training, evaluation, deployment, monitoring).
                Specifically suggest:
                1.  What parameters related to this tool could be logged to MLflow?
                2.  What metrics could be tracked?
                3.  What kind of artifacts could be logged (e.g., config files, model files specific to the tool, evaluation plots)?
                4.  Are there any known MLflow plugins or standard integration patterns? (Perform a quick search if unsure).
                5.  What are the key challenges or considerations for integration?

                Your final output must be a report addressing these 5 points clearly.
                Focus solely on the MLflow integration aspect.
            """
            ),
            agent=agent,
            expected_output="A report detailing potential MLflow integration points (params, metrics, artifacts), known integrations, and challenges.",
            context=[context_task] # Explicitly state dependency on the previous task
        )

print("MLflowIntegrationTasks class defined.")

MLflowIntegrationTasks class defined.


In [17]:
# --- Cell 6: Define the Crew Class (with MLflow additions) ---
import mlflow
import time
import json # Needed for logging dicts
from crewai import __version__ as crewai_version # Get CrewAI version

class MLflowIntegrationCrew:
    def __init__(self, area_of_interest):
        self.area_of_interest = area_of_interest
        # Determine LLM config (as before)
        self.llm_config = None # Default to env vars

        # Store agent/task definitions for logging
        self.agents_config = {}
        self.tasks_config = {}

    def _log_configs(self):
        """Logs agent and task configurations as artifacts."""
        print("Logging configurations to MLflow...")
        # Log agent configs
        for name, agent_obj in self.agents_config.items():
            # Convert Agent object to a dictionary (might need refinement based on Agent structure)
            # Simple example: extracting key attributes
            agent_dict = {
                "role": agent_obj.role,
                "goal": agent_obj.goal,
                "backstory": agent_obj.backstory,
                "tools": [tool.name for tool in agent_obj.tools] if agent_obj.tools else [],
                "llm": str(agent_obj.llm) if agent_obj.llm else "Default",
                "verbose": agent_obj.verbose,
                "allow_delegation": agent_obj.allow_delegation
            }
            mlflow.log_dict(agent_dict, f"agent_configs/{name}_config.json")

        # Log task configs
        for name, task_obj in self.tasks_config.items():
            task_dict = {
                "description": task_obj.description,
                "expected_output": task_obj.expected_output,
                "agent": task_obj.agent.role if task_obj.agent else "N/A", # Log agent role
                 # Context might be complex to serialize, log its task name/description if possible
                "context_task_descriptions": [ctx.description for ctx in task_obj.context] if task_obj.context else [],
            }
            mlflow.log_dict(task_dict, f"task_configs/{name}_config.json")

    def run(self):
        start_time = time.time()
        final_result = None
        status = "FAILURE" # Default status

        # --- MLflow Run Start ---
        # You might want to set experiment name outside the class or pass it in
        # mlflow.set_experiment("CrewAI MLflow Demo")
        with mlflow.start_run(run_name=f"CrewAI_Scout_{self.area_of_interest[:30]}") as run:
            print(f"MLflow Run started: {run.info.run_id}")

            # 1. Log Parameters & Tags
            print("Logging parameters and tags...")
            mlflow.log_param("area_of_interest", self.area_of_interest)
            mlflow.log_param("crew_process", "sequential") # Assuming sequential
            mlflow.log_param("crewai_version", crewai_version)
            mlflow.set_tag("crew_input_area", self.area_of_interest)
            mlflow.set_tag("crewai_version", crewai_version)
            # Add LLM info if available/consistent
            # llm_model_name = self.llm_config.model_name if self.llm_config else "Default"
            # mlflow.log_param("llm_model", llm_model_name)
            # mlflow.set_tag("llm_model", llm_model_name)

            # Instantiate agents & tasks, store their configs
            agent_creator = MLflowIntegrationAgents()
            task_creator = MLflowIntegrationTasks()

            self.agents_config = {
                "trend_scout": agent_creator.trend_scout_agent(self.llm_config),
                "tech_analyst": agent_creator.technical_analyst_agent(self.llm_config),
                "mlflow_assessor": agent_creator.mlflow_integration_assessor_agent(self.llm_config),
            }
            # Assign agents to local variables for clarity if needed
            trend_scout = self.agents_config["trend_scout"]
            tech_analyst = self.agents_config["tech_analyst"]
            mlflow_assessor = self.agents_config["mlflow_assessor"]

            # Define tasks sequentially and store configs
            scout_task = task_creator.scout_task(trend_scout, self.area_of_interest)
            analyze_task = task_creator.analyze_task(tech_analyst, self.area_of_interest, context_task=scout_task)
            assess_task = task_creator.assess_mlflow_integration_task(mlflow_assessor, context_task=analyze_task)

            self.tasks_config = {
                "scout_task": scout_task,
                "analyze_task": analyze_task,
                "assess_task": assess_task
            }

            # Log Agent/Task Counts
            mlflow.log_metric("agent_count", len(self.agents_config))
            mlflow.log_metric("task_count", len(self.tasks_config))
            mlflow.set_tag("crew_agents", "|".join(self.agents_config.keys()))

            # Log full configs as artifacts
            self._log_configs()

            # Form the crew
            crew = Crew(
                agents=list(self.agents_config.values()),
                tasks=list(self.tasks_config.values()),
                process=Process.sequential,
                verbose=True, # Keep verbose True to capture logs if needed
                # llm=self.llm_config # Optional crew-level LLM
            )

            # 2. Execute the Crew & Capture Output/Logs
            print("\nAttempting to kickoff the crew...")
            try:
                # --- Capture verbose output ---
                # This part can be tricky in standard notebooks.
                # Option A: Use a context manager if available (e.g., from libraries like `io`, `contextlib`)
                # Option B: Run as script and redirect stdout/stderr (easier)
                # Option C: Rely on parsing the 'final_result' if it contains verbose output (less common now)

                # Simple approach: run and hope final_result is useful, log separately if needed
                final_result = crew.kickoff()
                status = "SUCCESS"
                print("Crew kickoff successful.")

            except Exception as e:
                status = "FAILURE"
                final_result = f"Error Type: {type(e).__name__}\nError Details: {e}\n\nTraceback:\n{traceback.format_exc()}"
                print(f"\n--- Crew Execution Failed ---")
                print(final_result)
                # Optionally re-raise the exception if you want the notebook cell to fail
                # raise e
            finally:
                # 3. Log Results & Metrics
                end_time = time.time()
                execution_time = end_time - start_time
                print(f"Execution Time: {execution_time:.2f} seconds")
                print("Logging results and metrics...")

                mlflow.log_metric("execution_time_seconds", execution_time)
                mlflow.set_tag("crew_status", status)
                mlflow.log_metric("success", 1 if status == "SUCCESS" else 0)

                # Log final output as text artifact
                if isinstance(final_result, str):
                    mlflow.log_text(final_result, "final_output.txt")
                else:
                     # Try logging as JSON if it's dict-like, else convert to string
                    try:
                        mlflow.log_dict(dict(final_result), "final_output.json") # Requires result implement items()
                    except:
                         mlflow.log_text(str(final_result), "final_output.txt")

                # --- How to log intermediate steps/verbose log? ---
                # If running as a script redirecting output:
                # with open("execution_log.txt", "r") as f:
                #    mlflow.log_artifact("execution_log.txt")

                # Placeholder - Add logic here if you capture verbose logs some other way
                print("Placeholder: Add logic to capture and log verbose execution output if needed.")

                print(f"MLflow Run completed: {run.info.run_id}")

        return final_result, status # Return status along with result




Running the crew with MLflow Tracing...
MLflow Run started: 6e32160883d942f49e54bf3dcf5e1b45
Logging parameters and tags...
Logging configurations to MLflow...


2025/03/29 20:42:11 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/pydantic/fields.py:783: PydanticDeprecatedSince20: `min_items` is deprecated and will be removed, use `min_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/"



Attempting to kickoff the crew...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 39246e51-eb41-4510-9f00-c609dde9fb69                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b9f486f6-917c-4347-9c02-249b1c9648ab
       Status: Executing Task...

...

🚀 Crew: crew
├── 📋 Task: b9f486f6-917c-4347-9c02-249b1c9648ab
│      Assigned to: AI Technology Trend Scout
│      Status: ✅ Completed
│   └── 🤖 Agent: AI Technology Trend Scout
│           Status: ✅ Completed
│       └── 🔧 Used Search in a specific website (42)
├── 📋 Task: 91e3642a-aa6c-45da-9908-a78b9e4a953c
│      Assigned to: AI Technical Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: AI Technical Analyst
│           Status: ✅ Completed
│       └── 🔧 Used Search in a specific website (51)
└── 📋 Task: 51ceee2c-cd67-4215-bead-b07b1b48d417
       Assigned to: MLflow Integration Assessor
       Status: ✅ Completed
    └── 🤖 Agent: MLflow Integration Assessor
            Status: ✅ Completed
        └── 🔧 Used Search in a specific website (52)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 51ceee2c-cd67-4215-bead-b07b1b48d417                                                                     │
│  Agent: MLflow Integration Assessor                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 39246e51-eb41-4510-9f00-c609dde9fb69                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Crew kickoff successful.
Execution Time: 119.47 seconds
Logging results and metrics...
Placeholder: Add logic to capture and log verbose execution output if needed.
MLflow Run completed: 6e32160883d942f49e54bf3dcf5e1b45
🏃 View run CrewAI_Scout_Explainable AI (XAI) libraries at: https://public-tracking.mlflow-e00e997y0znyf15mwg.backbone-e00rt16p71ew33dg9e.msp.eu-north1.nebius.cloud/#/experiments/12/runs/6e32160883d942f49e54bf3dcf5e1b45
🧪 View experiment at: https://public-tracking.mlflow-e00e997y0znyf15mwg.backbone-e00rt16p71ew33dg9e.msp.eu-north1.nebius.cloud/#/experiments/12


########################
## Crew Run Status: SUCCESS ##
########################

Final Output (also logged to MLflow):
**Report on Integration of Explainable AI with MLflow**

1. **Parameters to Log to MLflow:**
   - **Model Parameters**: Configuration settings for transformer models such as number of layers, hidden units, dropout rate, and optimizer specifications.
   - **Explanation Method Parameters**: Setti

Trace(request_id=5648c411b9c641e3b0baffef1b2ef1a7)

### Run 

#### Suggested area Topics:

LLM Focused:

- "Tools for fine-tuning open-source LLMs" (e.g., Axolotl, Unsloth, libraries for PEFT)
- "Frameworks for evaluating LLM outputs" (e.g., Ragas, DeepEval, TruLens)
- "LLM Observability platforms" (e.g., LangSmith, Helicone, Weights & Biases integrations)
- "Prompt engineering & management toolkits" (e.g., Promptfoo, LangChain/LlamaIndex prompt templates, specialized libraries)
- "Quantization libraries for LLMs" (e.g., AutoGPTQ, bitsandbytes wrappers, specific toolkits)

RAG Focused:
- "Scalable Vector Databases for RAG" (e.g., Qdrant, Weaviate, Milvus alternatives or new entrants)
- "Frameworks implementing advanced RAG techniques" (e.g., Self-RAG, Corrective RAG, ReAct pattern libraries)
- "Evaluation frameworks specifically for RAG pipelines" (e.g., Ragas, TruLens focus on RAG metrics)

Agentic AI Focused:
- "Alternative AI Agent frameworks to LangChain/CrewAI" (e.g., Autogen, BabyAGI variants, new research frameworks)
- "Tools for creating and managing agent actions/tools" (Libraries simplifying tool definition and secure execution)
- "Frameworks for multi-agent system orchestration" (e.g., Autogen, specialized simulation environments)

General GenAI Dev Tools:
- "Orchestration tools for complex LLM/RAG pipelines" (e.g., Kestra, Dagster integrations for LLMs)
- "Synthetic data generation tools using LLMs" (Libraries focused on generating structured/unstructured data)

Tips for Choosing:
- Specificity: More specific topics often yield better results than very broad ones.
- Recency: Focus on areas where new tools are actively emerging.
- MLflow Relevance: While all can be tracked, areas like "Observability," "Evaluation," "Fine-tuning," and "Orchestration" have very direct conceptual links to MLflow's core purpose.


In [19]:
# --- Cell 8: Run the Crew and Display Results (Modified) ---
import traceback # Needed for logging exception tracebacks

print("Running the crew with MLflow Tracing...")
area = "Evaluation frameworks specifically for RAG pipelines"
integration_crew = MLflowIntegrationCrew(area_of_interest=area)

# Run and capture status
final_result, run_status = integration_crew.run()

print("\n\n########################")
print(f"## Crew Run Status: {run_status} ##")
print("########################\n")

print("Final Output (also logged to MLflow):")
print(final_result)

# You can now go to the MLflow UI (usually `mlflow ui` in terminal) to see the run details.

Running the crew with MLflow Tracing...
MLflow Run started: 5150278cd9af488596c2557c538da876
Logging parameters and tags...
Logging configurations to MLflow...

Attempting to kickoff the crew...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b17f0782-8e0e-495f-92a0-4b5f58e0fc0b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 5b0abf75-deea-4b40-9bc9-8ef60c78c6e6
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 5b0abf75-deea-4b40-9bc9-8ef60c78c6e6
       Status: Executing Task...
    └── 🤖 Agent: AI Technology Trend Scout
            Status: In Progress

# Agent: AI Technology Trend Scout
## Task: 
Identify one specific, noteworthy, and relatively new open-source tool,
library, or technique related to 'Evaluation frameworks specifically for RAG pipelines'.
Provide its name and a brief (1-2 sentence) description of what it does.
Focus on things gaining traction or offering novel capabilities.
Avoid general concepts; find a concrete example.

Example Areas: Vector Databases, LLM Serving Frameworks,
               Data Versioning for ML, Explainable AI Libraries.

Your final output MUST be the name of the tool/technique and the brief description ONLY.



...

🚀 Crew: crew
├── 📋 Task: 5b0abf75-deea-4b40-9bc9-8ef60c78c6e6
│      Assigned to: AI Technology Trend Scout
│      Status: ✅ Completed
│   └── 🤖 Agent: AI Technology Trend Scout
│           Status: ✅ Completed
│       └── 🔧 Used Search in a specific website (65)
├── 📋 Task: d5b68f74-015b-4891-bab5-7b4983bc98fb
│      Assigned to: AI Technical Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: AI Technical Analyst
│           Status: ✅ Completed
│       └── 🔧 Used Search in a specific website (72)
└── 📋 Task: 51acab0d-ccbb-40ef-84af-6a617122d999
       Assigned to: MLflow Integration Assessor
       Status: ✅ Completed
    └── 🤖 Agent: MLflow Integration Assessor
            Status: ✅ Completed
        └── 🔧 Used Search in a specific website (73)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 51acab0d-ccbb-40ef-84af-6a617122d999                                                                     │
│  Agent: MLflow Integration Assessor                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b17f0782-8e0e-495f-92a0-4b5f58e0fc0b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Crew kickoff successful.
Execution Time: 124.86 seconds
Logging results and metrics...
Placeholder: Add logic to capture and log verbose execution output if needed.
MLflow Run completed: 5150278cd9af488596c2557c538da876
🏃 View run CrewAI_Scout_Evaluation frameworks specific at: https://public-tracking.mlflow-e00e997y0znyf15mwg.backbone-e00rt16p71ew33dg9e.msp.eu-north1.nebius.cloud/#/experiments/12/runs/5150278cd9af488596c2557c538da876
🧪 View experiment at: https://public-tracking.mlflow-e00e997y0znyf15mwg.backbone-e00rt16p71ew33dg9e.msp.eu-north1.nebius.cloud/#/experiments/12


########################
## Crew Run Status: SUCCESS ##
########################

Final Output (also logged to MLflow):
**Report: Integration of the Evaluate Library with MLflow**

1. **Parameters Related to the Tool to be Logged to MLflow:**
   - `evaluation_metric`: The type of evaluation metric used (e.g., accuracy, F1 score, etc.).
   - `batch_size`: The size of the data batch during evaluation.
   - `model_

Trace(request_id=6b26c68c6c6f48b7bb15ee2086240490)

# Example 5: AI Stack Assistant 

In [8]:
# Turn on auto tracing by calling mlflow.crewai.autolog()
mlflow.crewai.autolog()

mlflow.set_experiment("AI-Stack")

2025/04/15 13:40:03 INFO mlflow.tracking.fluent: Experiment with name 'AI-Stack' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/4', creation_time=1744717203269, experiment_id='4', last_update_time=1744717203269, lifecycle_stage='active', name='AI-Stack', tags={}>

In [17]:
from crewai import Agent, Crew, Task, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from textwrap import dedent
import os

# Initialize search tools
search_tool = WebsiteSearchTool()
serper_tool = SerperDevTool()  # Requires SERPER_API_KEY in environment variables

class AIToolResearchAgents:
    def researcher_agent(self):
        """Agent responsible for discovering relevant AI tools for specific tasks."""
        return Agent(
            role="AI Tool Researcher",
            goal="Discover and evaluate relevant AI tools (libraries, frameworks) for specific tasks, considering the existing AI stack used in the project.",
            backstory=dedent("""
                You are an AI research specialist with extensive knowledge of the AI ecosystem. 
                Your expertise lies in finding the most appropriate tools for specific tasks, 
                considering compatibility with existing technologies. You excel at evaluating 
                open-source projects, understanding their purpose, and identifying their potential 
                use cases. You always back your recommendations with solid research.
            """),
            tools=[search_tool, serper_tool],
            verbose=True,
            allow_delegation=False,
            max_iter=3
        )

    def analyst_agent(self):
        """Agent responsible for analyzing tools and creating detailed reports."""
        return Agent(
            role="Technical Analyst",
            goal="Thoroughly analyze AI tools and prepare comprehensive reports highlighting strengths, weaknesses, and integration opportunities.",
            backstory=dedent("""
                You are a technical expert specializing in AI technologies evaluation. 
                You have a deep understanding of AI libraries and frameworks, and can 
                quickly assess their technical merits. Your analytical skills allow you 
                to identify pros and cons of each tool, and you're known for creating 
                clear, well-structured reports that help teams make informed decisions 
                about which technologies to adopt.
            """),
            tools=[search_tool, serper_tool],
            verbose=True,
            allow_delegation=False,
            max_iter=3
        )

class AIToolResearchTasks:
    def search_tools_task(self, agent, task, ai_stack):
        """Task to discover relevant tools for a specific task considering the existing AI stack."""
        return Task(
            description=dedent(f"""
                Research and identify 3-5 relevant open-source AI tools (libraries, frameworks) for the task: '{task}'.
                Consider compatibility with the existing AI stack: '{ai_stack}'.
                
                For each tool found, identify:
                - Name
                - Primary URL (GitHub repository or official website)
                - Primary use case
                - Brief description
                
                Your output should be formatted as a JSON list of objects:
                [
                  {{
                    "name": "Tool Name",
                    "url": "https://github.com/org/repo",
                    "use_case": "Primary use case in one sentence",
                    "description": "Brief description of the tool (2-3 sentences)"
                  }},
                  ...
                ]
                
                Focus on finding the most appropriate and well-maintained tools for the task.
                Consider GitHub stars, recent activity, documentation quality, and community support.
            """),
            agent=agent,
            expected_output="A JSON list containing details of 3-5 relevant AI tools with their names, URLs, use cases, and descriptions.",
            output_file="output/tool_candidates.json"
        )

    def analyze_tools_task(self, agent, task, ai_stack):
        """Task to perform in-depth analysis of discovered tools."""
        return Task(
            description=dedent(f"""
                Read the tool list from the previous task's result and perform a detailed analysis of each tool.
                
                For each tool:
                1. Research its capabilities, limitations, community adoption, documentation quality
                2. Evaluate how well it addresses the specified task: '{task}'
                3. Consider its compatibility with '{ai_stack}'
                4. Identify pros and cons
                
                Generate a detailed JSON analysis with these fields for each tool:
                - name: Tool name
                - url: Primary URL
                - description: Comprehensive description
                - features: List of key features
                - pros: List of advantages
                - cons: List of limitations
                - integration_complexity: "Low"/"Medium"/"High"
                - community_support: Assessment of community and maintenance
                - documentation_quality: Assessment of documentation
                - recommendation_score: 1-10 integer
                - recommendation_justification: Explanation of the score
                
                Your output should be a JSON list of these detailed analysis objects.
            """),
            agent=agent,
            expected_output="A JSON list containing detailed analysis of each tool with comprehensive information about features, pros, cons, and recommendation scores.",
            output_file="output/tool_analysis.json"
        )

    def create_report_task(self, agent, task, ai_stack):
        """Task to create a comprehensive report with recommendations."""
        return Task(
            description=dedent(f"""
                Read the analysis from the previous task's result and create a comprehensive Markdown report.
                
                The report should include:
                
                1. An introduction explaining the task ('{task}') and existing stack ('{ai_stack}')
                2. For each tool, create a section with:
                   - Tool name and URL as a heading
                   - Description
                   - Features (as bullet points)
                   - Pros (as bullet points)
                   - Cons (as bullet points)
                   - Integration complexity
                   - Recommendation score with justification
                3. A summary/conclusion comparing the tools and providing final recommendations
                
                Use proper Markdown formatting with headings, bullet points, and emphasis where appropriate.
                Sort tools by recommendation score (descending).
                
                Your output should be a complete, well-formatted Markdown document.
            """),
            agent=agent,
            expected_output="A comprehensive Markdown report analyzing each tool with recommendations, properly formatted with headings, bullet points, and clear sections.",
            output_file="output/tool_recommendation_report.md"
        )

class AIToolResearchCrew:
    def __init__(self, task, ai_stack):
        """
        Initialize the crew with the task description and existing AI stack.
        
        Args:
            task (str): Description of the task requiring AI tools
            ai_stack (str): Comma-separated list of existing tools/frameworks used
        """
        self.task = task
        self.ai_stack = ai_stack
        
        # Ensure output directory exists
        os.makedirs("output", exist_ok=True)

    def run(self):
        """Execute the research, analysis, and reporting process."""
        # Initialize agents
        agents = AIToolResearchAgents()
        researcher = agents.researcher_agent()
        analyst = agents.analyst_agent()
        
        # Initialize tasks
        tasks = AIToolResearchTasks()
        search_task = tasks.search_tools_task(researcher, self.task, self.ai_stack)
        
        # Create the crew first
        crew = Crew(
            agents=[researcher, analyst],
            tasks=[search_task],
            verbose=True,
            process=Process.sequential,
            memory=True
        )
        
        # Now add the dependent tasks that need context
        analyze_task = tasks.analyze_tools_task(analyst, self.task, self.ai_stack)
        crew.tasks.append(analyze_task)
        
        report_task = tasks.create_report_task(analyst, self.task, self.ai_stack)
        crew.tasks.append(report_task)
        
        result = crew.kickoff()
        return result

# # Example usage:
# if __name__ == "__main__":
#     task_description = "Develop a conversational RAG system that can answer questions based on a large PDF document collection"
#     existing_stack = "LangChain, OpenAI, Pinecone, FastAPI"
    
#     crew = AIToolResearchCrew(task_description, existing_stack)
#     result = crew.run()
    
#     print("\n=== Research Complete ===")
#     print(f"Check the output directory for detailed results.")
#     print("Final Report Path: output/tool_recommendation_report.md")

In [29]:
# Example usage:

task_description = "Develop a conversational RAG system that can answer questions based on a large PDF document collection"
existing_stack = "LangChain, OpenAI, Pinecone, FastAPI"

ai_dev_crew = AIToolResearchCrew(task_description, existing_stack)
result = ai_dev_crew.run()


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c9539c4a-cad7-4b35-82a4-67fabbc3aaf6                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b75ff9e7-9148-4e31-8350-a58228f0cba9
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b75ff9e7-9148-4e31-8350-a58228f0cba9
       Status: Executing Task...
    └── 🤖 Agent: AI Tool Researcher
            Status: In Progress

🤖 Agent: AI Tool Researcher
    Status: In Progress

🤖 Agent: AI Tool Researcher
    Status: In Progress
└── 🧠 Thinking...

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b7af42c8-7469-4ed9-b4fb-f60d771ab98c                                                                     │
│  Agent: Technical Analyst                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c9539c4a-cad7-4b35-82a4-67fabbc3aaf6                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-04-15 21:12:26,513 - 8447970368 - 2441714926.py-2441714926:319 - WARNING: Error logging JSON data from output/tool_candidates.json: Expecting value: line 1 column 1 (char 0)
2025-04-15 21:12:26,887 - 8447970368 - 2441714926.py-2441714926:319 - WARNING: Error logging JSON data from output/tool_analysis.json: Expecting value: line 1 column 1 (char 0)


🏃 View run Tool_Research_1744744212 at: https://public-tracking.mlflow-e00rfd6s4w3cdtk9fx.backbone-e00rt16p71ew33dg9e.msp.eu-north1.nebius.cloud/#/experiments/5/runs/5145b0c944ae4c16a9cb7e37da282df2
🧪 View experiment at: https://public-tracking.mlflow-e00rfd6s4w3cdtk9fx.backbone-e00rt16p71ew33dg9e.msp.eu-north1.nebius.cloud/#/experiments/5


Trace(request_id=1765b0f55cbc46ffb2dac139d858fc72)

In [19]:

print("\n=== Research Complete ===")
print(f"Check the output directory for detailed results.")
print("Final Report Path: output/tool_recommendation_report.md")


=== Research Complete ===
Check the output directory for detailed results.
Final Report Path: output/tool_recommendation_report.md


## Add "autolog" for LLM calls tracing

## Organize Runs and add artifacts tracking

In [25]:
from crewai import Agent, Crew, Task, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from textwrap import dedent
import os
import mlflow
import json
import time
import logging
import io
import sys
import contextlib

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Initialize search tools
search_tool = WebsiteSearchTool()
serper_tool = SerperDevTool()  # Requires SERPER_API_KEY in environment variables

class AIToolResearchAgents:
    def researcher_agent(self):
        """Agent responsible for discovering relevant AI tools for specific tasks."""
        return Agent(
            role="AI Tool Researcher",
            goal="Discover and evaluate relevant AI tools (libraries, frameworks) for specific tasks, considering the existing AI stack used in the project.",
            backstory=dedent("""
                You are an AI research specialist with extensive knowledge of the AI ecosystem. 
                Your expertise lies in finding the most appropriate tools for specific tasks, 
                considering compatibility with existing technologies. You excel at evaluating 
                open-source projects, understanding their purpose, and identifying their potential 
                use cases. You always back your recommendations with solid research.
            """),
            tools=[search_tool, serper_tool],
            verbose=True,
            allow_delegation=False,
            max_iter=3
        )

    def analyst_agent(self):
        """Agent responsible for analyzing tools and creating detailed reports."""
        return Agent(
            role="Technical Analyst",
            goal="Thoroughly analyze AI tools and prepare comprehensive reports highlighting strengths, weaknesses, and integration opportunities.",
            backstory=dedent("""
                You are a technical expert specializing in AI technologies evaluation. 
                You have a deep understanding of AI libraries and frameworks, and can 
                quickly assess their technical merits. Your analytical skills allow you 
                to identify pros and cons of each tool, and you're known for creating 
                clear, well-structured reports that help teams make informed decisions 
                about which technologies to adopt.
            """),
            tools=[search_tool, serper_tool],
            verbose=True,
            allow_delegation=False,
            max_iter=3
        )

class AIToolResearchTasks:
    def search_tools_task(self, agent, task, ai_stack):
        """Task to discover relevant tools for a specific task considering the existing AI stack."""
        return Task(
            description=dedent(f"""
                Research and identify 3-5 relevant open-source AI tools (libraries, frameworks) for the task: '{task}'.
                Consider compatibility with the existing AI stack: '{ai_stack}'.
                
                For each tool found, identify:
                - Name
                - Primary URL (GitHub repository or official website)
                - Primary use case
                - Brief description
                
                Your output should be formatted as a JSON list of objects:
                [
                  {{
                    "name": "Tool Name",
                    "url": "https://github.com/org/repo",
                    "use_case": "Primary use case in one sentence",
                    "description": "Brief description of the tool (2-3 sentences)"
                  }},
                  ...
                ]
                
                Focus on finding the most appropriate and well-maintained tools for the task.
                Consider GitHub stars, recent activity, documentation quality, and community support.
            """),
            agent=agent,
            expected_output="A JSON list containing details of 3-5 relevant AI tools with their names, URLs, use cases, and descriptions.",
            output_file="output/tool_candidates.json"
        )

    def analyze_tools_task(self, agent, task, ai_stack):
        """Task to perform in-depth analysis of discovered tools."""
        return Task(
            description=dedent(f"""
                Read the tool list from the previous task's result and perform a detailed analysis of each tool.
                
                For each tool:
                1. Research its capabilities, limitations, community adoption, documentation quality
                2. Evaluate how well it addresses the specified task: '{task}'
                3. Consider its compatibility with '{ai_stack}'
                4. Identify pros and cons
                
                Generate a detailed JSON analysis with these fields for each tool:
                - name: Tool name
                - url: Primary URL
                - description: Comprehensive description
                - features: List of key features
                - pros: List of advantages
                - cons: List of limitations
                - integration_complexity: "Low"/"Medium"/"High"
                - community_support: Assessment of community and maintenance
                - documentation_quality: Assessment of documentation
                - recommendation_score: 1-10 integer
                - recommendation_justification: Explanation of the score
                
                Your output should be a JSON list of these detailed analysis objects.
            """),
            agent=agent,
            expected_output="A JSON list containing detailed analysis of each tool with comprehensive information about features, pros, cons, and recommendation scores.",
            output_file="output/tool_analysis.json"
        )

    def create_report_task(self, agent, task, ai_stack):
        """Task to create a comprehensive report with recommendations."""
        return Task(
            description=dedent(f"""
                Read the analysis from the previous task's result and create a comprehensive Markdown report.
                
                The report should include:
                
                1. An introduction explaining the task ('{task}') and existing stack ('{ai_stack}')
                2. For each tool, create a section with:
                   - Tool name and URL as a heading
                   - Description
                   - Features (as bullet points)
                   - Pros (as bullet points)
                   - Cons (as bullet points)
                   - Integration complexity
                   - Recommendation score with justification
                3. A summary/conclusion comparing the tools and providing final recommendations
                
                Use proper Markdown formatting with headings, bullet points, and emphasis where appropriate.
                Sort tools by recommendation score (descending).
                
                Your output should be a complete, well-formatted Markdown document.
            """),
            agent=agent,
            expected_output="A comprehensive Markdown report analyzing each tool with recommendations, properly formatted with headings, bullet points, and clear sections.",
            output_file="output/tool_recommendation_report.md"
        )

class AIToolResearchCrew:
    def __init__(self, task, ai_stack):
        """
        Initialize the crew with the task description and existing AI stack.
        
        Args:
            task (str): Description of the task requiring AI tools
            ai_stack (str): Comma-separated list of existing tools/frameworks used
        """
        self.task = task
        self.ai_stack = ai_stack
        self.run_id = None
        self.execution_log = io.StringIO()
        self.crew = None
        
        # Ensure output directory exists
        os.makedirs("output", exist_ok=True)

    def run(self):
        """Execute the research, analysis, and reporting process with MLflow tracking."""
        # Set up MLflow
        mlflow.set_experiment("AI Tool Research Crew")
        
        # Start MLflow run
        with mlflow.start_run(run_name=f"Tool_Research_{int(time.time())}") as mlflow_run:
            self.run_id = mlflow_run.info.run_id
            
            # Log parameters
            mlflow.log_param("task", self.task)
            mlflow.log_param("ai_stack", self.ai_stack)
            
            # Capture console output
            with contextlib.redirect_stdout(self.execution_log), contextlib.redirect_stderr(self.execution_log):
                try:
                    # Initialize agents
                    logger.info("Initializing agents...")
                    agents = AIToolResearchAgents()
                    researcher = agents.researcher_agent()
                    analyst = agents.analyst_agent()
                    
                    # Log agent configurations
                    mlflow.log_dict(
                        {
                            "researcher": {
                                "role": researcher.role,
                                "goal": researcher.goal,
                                "backstory": researcher.backstory,
                                "allow_delegation": researcher.allow_delegation,
                                "max_iter": researcher.max_iter
                            },
                            "analyst": {
                                "role": analyst.role,
                                "goal": analyst.goal,
                                "backstory": analyst.backstory,
                                "allow_delegation": analyst.allow_delegation,
                                "max_iter": analyst.max_iter
                            }
                        },
                        "agent_configurations.json"
                    )
                    
                    # Track execution time
                    start_time = time.time()
                    
                    # Initialize tasks
                    logger.info("Setting up tasks...")
                    tasks = AIToolResearchTasks()
                    search_task = tasks.search_tools_task(researcher, self.task, self.ai_stack)
                    
                    # Create the crew
                    logger.info("Creating crew...")
                    crew = Crew(
                        agents=[researcher, analyst],
                        tasks=[search_task],
                        verbose=True,
                        process=Process.sequential,
                        memory=True
                    )
                    
                    # Add dependent tasks
                    analyze_task = tasks.analyze_tools_task(analyst, self.task, self.ai_stack)
                    crew.tasks.append(analyze_task)
                    
                    report_task = tasks.create_report_task(analyst, self.task, self.ai_stack)
                    crew.tasks.append(report_task)
                    
                    # Start the crew
                    logger.info("Starting crew execution...")
                    result = crew.kickoff()
                    logger.info("Crew execution completed.")
                    
                    # Log execution time
                    execution_time = time.time() - start_time
                    mlflow.log_metric("execution_time_seconds", execution_time)
                    
                    # Log task artifacts if they exist
                    self._log_artifacts()
                    
                    # Set success tag
                    if os.path.exists("output/tool_recommendation_report.md"):
                        mlflow.set_tag("status", "SUCCESS")
                        mlflow.log_metric("success", 1)
                    else:
                        mlflow.set_tag("status", "FAILED")
                        mlflow.log_metric("success", 0)
                    
                    # Log crew metrics
                    if hasattr(crew, "usage_metrics") and crew.usage_metrics:
                        metrics = crew.usage_metrics
                        if hasattr(metrics, "total_tokens"):
                            mlflow.log_metric("total_tokens", metrics.total_tokens)
                        if hasattr(metrics, "prompt_tokens"):
                            mlflow.log_metric("prompt_tokens", metrics.prompt_tokens)
                        if hasattr(metrics, "completion_tokens"):
                            mlflow.log_metric("completion_tokens", metrics.completion_tokens)
                        if hasattr(metrics, "total_cost"):
                            mlflow.log_metric("total_cost", metrics.total_cost)

                    self.crew = crew
                    
                    return result
                
                except Exception as e:
                    logger.error(f"Error during crew execution: {str(e)}", exc_info=True)
                    mlflow.set_tag("status", "FAILED")
                    mlflow.log_metric("success", 0)
                    raise e
                
                finally:
                    # Log execution trace
                    logger.info("Logging execution trace...")
                    execution_log = self.execution_log.getvalue()
                    mlflow.log_text(execution_log, "execution_log.txt")

    def _log_artifacts(self):
        """Log task output artifacts to MLflow."""
        artifact_files = [
            ("output/tool_candidates.json", "Task 1: Tool Discovery"),
            ("output/tool_analysis.json", "Task 2: Tool Analysis"),
            ("output/tool_recommendation_report.md", "Task 3: Final Report")
        ]
        
        for file_path, description in artifact_files:
            if os.path.exists(file_path):
                logger.info(f"Logging artifact: {file_path}")
                mlflow.log_artifact(file_path)
                
                # For JSON files, also log as parameters for easier viewing
                if file_path.endswith('.json'):
                    try:
                        with open(file_path, 'r') as f:
                            data = json.load(f)
                            
                        if file_path == "output/tool_candidates.json":
                            # Log tool names as parameters
                            tool_names = [tool.get('name', f"Tool {i+1}") for i, tool in enumerate(data)]
                            mlflow.log_param("discovered_tools", ", ".join(tool_names))
                            
                        elif file_path == "output/tool_analysis.json":
                            # Log recommendation scores as metrics
                            for i, tool in enumerate(data):
                                tool_name = tool.get('name', f"Tool {i+1}")
                                score = tool.get('recommendation_score', 0)
                                if isinstance(score, (int, float)):
                                    mlflow.log_metric(f"{tool_name}_score", score)
                    except Exception as e:
                        logger.warning(f"Error logging JSON data from {file_path}: {str(e)}")
            else:
                logger.warning(f"Artifact file not found: {file_path}")

# Example usage:
# if __name__ == "__main__":
    # task_description = "Develop a conversational RAG system that can answer questions based on a large PDF document collection"
    # existing_stack = "LangChain, OpenAI, Pinecone, FastAPI"
    
    # crew = AIToolResearchCrew(task_description, existing_stack)
    # result = crew.run()
    
    # print("\n=== Research Complete ===")
    # print(f"Check the output directory for detailed results.")
    # print("Final Report Path: output/tool_recommendation_report.md")
    # print(f"MLflow Run ID: {crew.run_id}")
    # print(f"View details at: http://localhost:5000/#/experiments/1/runs/{crew.run_id}")

In [30]:
# Let's run it once without MLflow to see the typical verbose console output.

# Turn on auto tracing by calling mlflow.crewai.autolog()
mlflow.crewai.autolog()
mlflow.set_experiment("Autolog")

ai_dev_crew = AIOpsResearchCrew(task_description, existing_stack)
result = ai_dev_crew.run()

NameError: name 'AIOpsResearchCrew' is not defined

In [ ]:
ai_dev_crew

In [26]:
task_description = "Develop a conversational RAG system that can answer questions based on a large PDF document collection"
existing_stack = "LangChain, OpenAI, Pinecone, FastAPI"

crew = AIToolResearchCrew(task_description, existing_stack)
result = crew.run()


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e436f71c-f10a-4dcc-a65b-9ba87906817b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 1e4e360b-4ba0-459c-9818-7a23640f3766
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 1e4e360b-4ba0-459c-9818-7a23640f3766
       Status: Executing Task...
    └── 🤖 Agent: AI Tool Researcher
            Status: In Progress

🤖 Agent: AI Tool Researcher
    Status: In Progress

🤖 Agent: AI Tool Researcher
    Status: In Progress
└── 🧠 Thinking...

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 38936bdf-6a3c-4b02-ad9e-b780a87edd73                                                                     │
│  Agent: Technical Analyst                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e436f71c-f10a-4dcc-a65b-9ba87906817b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-04-15 13:58:16,299 - 8447970368 - 2441714926.py-2441714926:319 - WARNING: Error logging JSON data from output/tool_analysis.json: Expecting value: line 1 column 1 (char 0)


🏃 View run Tool_Research_1744718194 at: https://public-tracking.mlflow-e00rfd6s4w3cdtk9fx.backbone-e00rt16p71ew33dg9e.msp.eu-north1.nebius.cloud/#/experiments/5/runs/fd43eb499a324e8190f976820803fb7d
🧪 View experiment at: https://public-tracking.mlflow-e00rfd6s4w3cdtk9fx.backbone-e00rt16p71ew33dg9e.msp.eu-north1.nebius.cloud/#/experiments/5


Trace(request_id=d27f2569e0ff4a0694b622fd36b3abe5)

In [27]:
print("\n=== Research Complete ===")
print(f"Check the output directory for detailed results.")
print("Final Report Path: output/tool_recommendation_report.md")
print(f"MLflow Run ID: {crew.run_id}")


=== Research Complete ===
Check the output directory for detailed results.
Final Report Path: output/tool_recommendation_report.md
MLflow Run ID: fd43eb499a324e8190f976820803fb7d


## Add metrics

In [28]:
# crew = AIToolResearchCrew(task_description, existing_stack)
# result = crew.kickoff()

print(crew.usage_metrics)

AttributeError: 'AIToolResearchCrew' object has no attribute 'usage_metrics'

## Update prompts

## Evaluate quality of the Agent

In [ ]:
import os
import json
import time
import logging
import io
import sys
import contextlib

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


class AIOpsResearchCrew:
    def __init__(self, task, ai_stack, llm_config=None, evaluator_llm_config=None):
        """
        Initialize the crew with the task description and existing AI stack.
        
        Args:
            task (str): Description of the task requiring AI tools
            ai_stack (str): Comma-separated list of existing tools/frameworks used
            llm_config (Optional): LLM configuration to use for the agents
            evaluator_llm_config (Optional): LLM configuration to use for the evaluator (defaults to strong model)
        """
        self.task = task
        self.ai_stack = ai_stack
        self.llm_config = llm_config
        self.evaluator_llm_config = evaluator_llm_config
        self.run_id = None
        
        # Get LLM model name for logging
        self.llm_name = getattr(llm_config, "model_name", "default") if llm_config else "default"
        
        # Ensure output directory exists
        os.makedirs("output", exist_ok=True)
        
        # Initialize MLflow
        mlflow.set_experiment("AIOps Tool Research")

    def run(self, evaluate=True):
        """
        Execute the research, analysis, and reporting process.
        
        Args:
            evaluate (bool): Whether to evaluate the final report
            
        Returns:
            The result from the crew execution
        """
        # Start MLflow tracking
        with mlflow.start_run(run_name=f"Tool_Research_{self.llm_name}_{int(time.time())}") as run:
            self.run_id = run.info.run_id
            
            # Log parameters
            mlflow.log_param("task", self.task)
            mlflow.log_param("ai_stack", self.ai_stack)
            mlflow.log_param("llm_model", self.llm_name)
            
            # Capture logs
            log_buffer = io.StringIO()
            start_time = time.time()
            
            with contextlib.redirect_stdout(log_buffer), contextlib.redirect_stderr(log_buffer):
                try:
                    # Initialize agents
                    agents = AIOpsResearchAgents()
                    researcher = agents.researcher_agent(self.llm_config)
                    analyst = agents.analyst_agent(self.llm_config)
                    
                    # Initialize tasks
                    tasks = AIOpsResearchTasks()
                    search_task = tasks.search_tools_task(researcher, self.task, self.ai_stack)
                    
                    # Create the crew
                    crew = Crew(
                        agents=[researcher, analyst],
                        tasks=[search_task],
                        verbose=True,
                        process=Process.sequential,
                        memory=True
                    )
                    
                    # Add dependent tasks
                    analyze_task = tasks.analyze_tools_task(analyst, self.task, self.ai_stack)
                    crew.tasks.append(analyze_task)
                    
                    report_task = tasks.create_report_task(analyst, self.task, self.ai_stack)
                    crew.tasks.append(report_task)
                    
                    # Execute the crew
                    result = crew.kickoff()
                    
                    # Log artifacts
                    if os.path.exists("output/tool_candidates.json"):
                        mlflow.log_artifact("output/tool_candidates.json")
                    
                    if os.path.exists("output/tool_analysis.json"):
                        mlflow.log_artifact("output/tool_analysis.json")
                        
                    if os.path.exists("output/tool_recommendation_report.md"):
                        mlflow.log_artifact("output/tool_recommendation_report.md")
                        mlflow.set_tag("status", "SUCCESS")
                    else:
                        mlflow.set_tag("status", "FAILED")
                    
                    # Evaluate the report if requested
                    if evaluate and os.path.exists("output/tool_recommendation_report.md"):
                        # Create evaluator agent with a strong model
                        evaluator = agents.evaluator_agent()
                        
                        # Create and run evaluation task
                        eval_task = tasks.evaluate_report_task(evaluator, self.task, self.ai_stack)
                        eval_crew = Crew(
                            agents=[evaluator],
                            tasks=[eval_task],
                            verbose=True
                        )
                        eval_result = eval_crew.kickoff()
                        
                        # Log evaluation results
                        if os.path.exists("output/report_evaluation.json"):
                            mlflow.log_artifact("output/report_evaluation.json")
                            
                            # Also log scores as metrics for easier comparison
                            try:
                                with open("output/report_evaluation.json", 'r') as f:
                                    eval_data = json.load(f)
                                    
                                for score_name, score_value in eval_data.items():
                                    if score_name.endswith('_score'):
                                        mlflow.log_metric(score_name, score_value)
                                
                                # Log reasoning as text
                                if 'reasoning' in eval_data:
                                    mlflow.log_text(eval_data['reasoning'], "evaluation_reasoning.txt")
                            except Exception as e:
                                logger.error(f"Error logging evaluation metrics: {str(e)}")
                    
                    # Log execution time
                    execution_time = time.time() - start_time
                    mlflow.log_metric("execution_time_seconds", execution_time)
                    
                    # Log usage metrics if available
                    if hasattr(crew, "usage_metrics") and crew.usage_metrics:
                        metrics = crew.usage_metrics
                        if hasattr(metrics, "total_tokens"):
                            mlflow.log_metric("total_tokens", metrics.total_tokens)
                        if hasattr(metrics, "total_cost"):
                            mlflow.log_metric("total_cost", metrics.total_cost)
                    
                    return result
                    
                except Exception as e:
                    logger.error(f"Error executing crew: {str(e)}", exc_info=True)
                    mlflow.set_tag("status", "ERROR")
                    mlflow.log_text(str(e), "error.txt")
                    raise e
                
                finally:
                    # Always log the execution logs
                    mlflow.log_text(log_buffer.getvalue(), "execution_log.txt")

In [ ]:
import os
import json
import time
import logging
import io
import sys
import contextlib

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


class AIOpsResearchCrew:
    def __init__(self, task, ai_stack):
        """
        Initialize the crew with the task description and existing AI stack.
        
        Args:
            task (str): Description of the task requiring AI tools
            ai_stack (str): Comma-separated list of existing tools/frameworks used
        """
        self.task = task
        self.ai_stack = ai_stack
        self.run_id = None
        self.execution_log = io.StringIO()
        self.crew = None
        
        # Ensure output directory exists
        os.makedirs("output", exist_ok=True)

    def run(self):
        """Execute the research, analysis, and reporting process with MLflow tracking."""
        
        # --- UPDATE: Start MLflow run ---
        with mlflow.start_run(run_name=f"Tool_Research_{int(time.time())}") as mlflow_run:
            self.run_id = mlflow_run.info.run_id
            
            # --- UPDATE: Log parameters ---
            mlflow.log_param("task", self.task)
            mlflow.log_param("ai_stack", self.ai_stack)
            
            # --- UPDATE: Capture console output ---
            with contextlib.redirect_stdout(self.execution_log), contextlib.redirect_stderr(self.execution_log):
                try:
                    # Initialize agents
                    logger.info("Initializing agents...")
                    agents = AIOpsResearchAgents()
                    researcher = agents.researcher_agent()
                    analyst = agents.analyst_agent()
                    
                    # Initialize tasks
                    logger.info("Setting up tasks...")
                    tasks = AIOpsResearchTasks()
                    search_task = tasks.search_tools_task(researcher, self.task, self.ai_stack)
                    
                    # Create the crew
                    logger.info("Creating crew...")
                    crew = Crew(
                        agents=[researcher, analyst],
                        tasks=[search_task],
                        verbose=True,
                        process=Process.sequential,
                        memory=True
                    )
                    self.crew = crew
                    
                    # Add dependent tasks
                    analyze_task = tasks.analyze_tools_task(analyst, self.task, self.ai_stack)
                    crew.tasks.append(analyze_task)
                    
                    report_task = tasks.create_report_task(analyst, self.task, self.ai_stack)
                    crew.tasks.append(report_task)
                    
                    # Start the crew
                    logger.info("Starting crew execution...")
                    result = crew.kickoff()
  
                    # --- UPDATE: Log metrics ---
                    logger.info("Starting log metrics...")
                    if hasattr(crew, "usage_metrics") and crew.usage_metrics:
                        mlflow.log_metrics(json.loads(crew.usage_metrics.json()))

                    
                    # --- UPDATE: Log task artifacts if they exist ---
                    self._log_artifacts()

                    # --- UPDATE: Set success tag ---
                    if os.path.exists("output/tool_recommendation_report.md"):
                        mlflow.set_tag("status", "SUCCESS")
                    else:
                        mlflow.set_tag("status", "FAILED")
                    
                    return result
                
                # --- UPDATE: Log execution status and logs 
                except Exception as e:
                    logger.error(f"Error during crew execution: {str(e)}", exc_info=True)
                    mlflow.set_tag("status", "FAILED")
                    mlflow.log_metric("success", 0)
                    raise e
                
                finally:
                    # Log execution trace
                    logger.info("Logging execution trace...")
                    execution_log = self.execution_log.getvalue()
                    mlflow.log_text(execution_log, "execution_log.txt")

    def _log_artifacts(self):
        """Log task output artifacts to MLflow."""
        artifact_files = [
            ("output/tool_candidates.json", "Task 1: Tool Discovery"),
        ]
        
        for file_path, description in artifact_files:
            if os.path.exists(file_path):
                logger.info(f"Logging artifact: {file_path}")
                mlflow.log_artifact(file_path)
                
                # For JSON files, also log as parameters for easier viewing
                if file_path.endswith('.json'):
                    try:
                        with open(file_path, 'r') as f:
                            data = json.load(f)
                            
                        # Log tool names as parameters
                        tool_names = [tool.get('name', f"Tool {i+1}") for i, tool in enumerate(data)]
                        mlflow.log_param("discovered_tools", ", ".join(tool_names))

                    except Exception as e:
                        logger.warning(f"Error logging JSON data from {file_path}: {str(e)}")
            else:
                logger.warning(f"Artifact file not found: {file_path}")